In [2]:
install.packages('data.table')

also installing the dependency 'chron'



package 'chron' successfully unpacked and MD5 sums checked
package 'data.table' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\568377\AppData\Local\Temp\RtmpgtMrxN\downloaded_packages
